# 3D Convex Hull

This notebook demonstrates the **Convex Hull** algorithm in 3D space.
While the 2D version creates a polygon enclosing the points, the 3D version creates a **convex polyhedron** (a mesh) that encloses all the points.

## Concept

Imagine a cloud of points in 3D space. The convex hull is the smallest convex shape that contains all these points. You can visualize it as:
*   **Shrink-wrapping** the points.
*   Stretching a rubber balloon around the points and letting it snap tight.

The resulting shape is a mesh composed of triangular faces.
### Why no self-intersections?
By definition, a **convex** object has no indentations or "caves". If you take any two points inside a convex object, the straight line connecting them stays entirely inside the object.
Because the algorithm always seeks the outermost boundary that satisfies this convexity rule, it is mathematically impossible for the resulting mesh to fold over itself or self-intersect. It guarantees a clean, closed, and valid mesh.


In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from scipy.spatial import ConvexHull


## 1. Generate Random 3D Points

We generate a cloud of random points in a 3D box.

In [ ]:
def generate_points_3d(n=50, bounds=(0, 100)):
    # Generate random points as a NumPy array of shape (n, 3)
    points = np.random.uniform(low=bounds[0], high=bounds[1], size=(n, 3))
    return points

# Generate 50 points
points = generate_points_3d(50)
print(f"Generated {len(points)} points.")
print(f"First 3 points:\n{points[:3]}")

## 2. Compute 3D Convex Hull (using Scipy)

We use `scipy.spatial.ConvexHull`, which implements the **Quickhull** algorithm.
This is a standard library for scientific computing in Python and does not require COMPAS.

**How Quickhull works (simplified):**
1.  Find extreme points (min/max x, y, z) to form an initial simplex (tetrahedron).
2.  For each face of the simplex, find the point furthest away from it (on the outside).
3.  "Expand" the hull to include that point, creating new faces and removing internal ones.
4.  Repeat until no points are outside the hull.

In [ ]:
# Compute Convex Hull
hull = ConvexHull(points)

# hull.simplices contains the indices of the points that form the triangular faces of the hull
print(f"Hull has {len(hull.vertices)} vertices and {len(hull.simplices)} faces.")
print(f"Indices of points on the hull: {hull.vertices}")

## 3. Visualize Result (Matplotlib)

We visualize the original point cloud and the resulting convex hull using `matplotlib`.

In [ ]:
def plot_hull_step(all_points, subset_points, title="Hull Step"):
    """
    Plots the convex hull of 'subset_points' while showing 'all_points' as ghosted.
    """
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot all points (Ghosted)
    ax.scatter(all_points[:,0], all_points[:,1], all_points[:,2], c='blue', marker='.', alpha=0.1, label='All Points')
    
    # Plot Subset points (Active)
    ax.scatter(subset_points[:,0], subset_points[:,1], subset_points[:,2], c='red', marker='o', alpha=1.0, label='Hull Vertices')
    
    # Compute Hull of the subset
    if len(subset_points) >= 4: # Need at least 4 points for 3D hull
        try:
            hull = ConvexHull(subset_points)
            
            # Plot Hull Faces
            polys = []
            for simplex in hull.simplices:
                face_points = subset_points[simplex]
                polys.append(face_points)
                
            mesh = Poly3DCollection(polys, alpha=0.5, edgecolor='k')
            mesh.set_facecolor('orange')
            ax.add_collection3d(mesh)
        except Exception as e:
            print(f"Could not compute hull: {e}")

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title(title)
    plt.legend()
    plt.show()

# --- Step 1: Initialization (Extreme Points) ---
# Find 6 extreme points (Min/Max X, Y, Z) to form the initial shape
# This mimics the first step of Quickhull
indices = set()
indices.add(np.argmin(points[:,0])) # Min X
indices.add(np.argmax(points[:,0])) # Max X
indices.add(np.argmin(points[:,1])) # Min Y
indices.add(np.argmax(points[:,1])) # Max Y
indices.add(np.argmin(points[:,2])) # Min Z
indices.add(np.argmax(points[:,2])) # Max Z

# Convert to list and get points
init_indices = list(indices)
current_hull_points = points[init_indices]

plot_hull_step(points, current_hull_points, title="Step 1: Initial Simplex (Extreme Points)")

# --- Step 2: Expansion (Add a distant point) ---
# In Quickhull, we look for points outside the current faces.
# For visualization, let's just pick a point that is NOT in our current set
# and is far from the center, to show the hull "growing".

# Simple centroid of current hull
centroid = np.mean(current_hull_points, axis=0)
# Find point furthest from centroid that is not yet in hull
dists = np.linalg.norm(points - centroid, axis=1)
# Sort indices by distance descending
sorted_indices = np.argsort(dists)[::-1]

# Find the first point not in our current set
for idx in sorted_indices:
    if idx not in init_indices:
        new_point_idx = idx
        break

# Add this point
step2_points = np.vstack([current_hull_points, points[new_point_idx]])
plot_hull_step(points, step2_points, title="Step 2: Expansion (Added furthest point)")

# --- Step 3: Final Result ---
# The final hull uses all points (Scipy filters the internal ones automatically)
plot_hull_step(points, points, title="Step 3: Final Convex Hull")

## COMPAS Alternative

While `scipy` is great for calculation, COMPAS provides geometry processing and visualization tools tailored for architecture.
Here is how to achieve the same result using COMPAS.

In [ ]:
from compas.geometry import convex_hull_numpy
from compas.topology import unify_cycles
from compas.datastructures import Mesh
from compas.colors import Color
from compas_notebook.viewer import Viewer

vertices, faces = convex_hull_numpy(points)

# Reindex vertices and faces
i_index = {i: index for index, i in enumerate(vertices)}
vertices = [points[index] for index in vertices]
faces = [[i_index[i] for i in face] for face in faces]
unify_cycles(vertices, faces)

# create mesh
mesh = Mesh.from_vertices_and_faces(vertices, faces)

# visualize
viewer = Viewer()
viewer.scene.add(mesh)
viewer.show()